In [ ]:
!pip install google-cloud-documentai

In [4]:
from google.cloud import documentai_v1 as documentai
import re
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage
import requests
from google.auth.transport.requests import Request
from google.oauth2 import service_account


### setup env values

In [8]:
# Set your config
project_id = "intelia-health"
location = "us"  # or your region
processor_id ="42c7c4ce5c8d8260"# 

gcs_uri = "gs://health-demo-pdf/Careplan/Download_Sample-Care-Plan_1.pdf"
DATASET_ID = "Whealth"
FHIR_STORE_ID = "whealth_fhirdata"
gcs_output_uri = "gs://health-demo-pdf/Careplan_output/"


fhir_store =f"projects/{project_id}/locations/{location}/datasets/{DATASET_ID}/fhirStores/{FHIR_STORE_ID}"


### extract text from pdf

In [9]:

from google.cloud import documentai_v1 as documentai

def extract_text_from_pdf(project_id, location, processor_id, gcs_input_uri, gcs_output_uri):
    client = documentai.DocumentProcessorServiceClient()
    
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    gcs_document = documentai.GcsDocument(
        gcs_uri=gcs_input_uri,
        mime_type="application/pdf"
    )

    input_documents = documentai.BatchDocumentsInputConfig(
        gcs_documents=documentai.GcsDocuments(documents=[gcs_document])
    )

    output_config = documentai.DocumentOutputConfig(
        gcs_output_config=documentai.DocumentOutputConfig.GcsOutputConfig(
            gcs_uri=gcs_output_uri
        )
    )

    request = documentai.BatchProcessRequest(
        name=name,
        input_documents=input_documents,
        document_output_config=output_config
    )

    operation = client.batch_process_documents(request=request)
 
    print("Waiting for operation to complete...")
    operation.result(timeout=300)
    print("Processing complete. Check GCS output folder.")
    return operation

### extract text, export it into gcs and read it from gcs

In [10]:
# Step 1: Extract text
text = extract_text_from_pdf(project_id,location, processor_id, gcs_uri,gcs_output_uri)

Waiting for operation to complete...
Processing complete. Check GCS output folder.


In [11]:
#get the extracted texts
import re
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage

operation=text
totaldoc=''
if 1==1:
    # After the operation is complete,
    # get output document information from operation metadata
    metadata = documentai.BatchProcessMetadata(operation.metadata)

    if metadata.state != documentai.BatchProcessMetadata.State.SUCCEEDED:
        raise ValueError(f"Batch Process Failed: {metadata.state_message}")

    storage_client = storage.Client()

    print("Output files:")
    # One process per Input Document
    for process in list(metadata.individual_process_statuses):
        # output_gcs_destination format: gs://BUCKET/PREFIX/OPERATION_NUMBER/INPUT_FILE_NUMBER/
        # The Cloud Storage API requires the bucket name and URI prefix separately
        matches = re.match(r"gs://(.*?)/(.*)", process.output_gcs_destination)
        if not matches:
            print(
                "Could not parse output GCS destination:",
                process.output_gcs_destination,
            )
            continue

        output_bucket, output_prefix = matches.groups()

        # Get List of Document Objects from the Output Bucket
        output_blobs = storage_client.list_blobs(output_bucket, prefix=output_prefix)

        # Document AI may output multiple JSON files per source file
        for blob in output_blobs:
            # Document AI should only output JSON files to GCS
            if blob.content_type != "application/json":
                print(
                    f"Skipping non-supported file: {blob.name} - Mimetype: {blob.content_type}"
                )
                continue

            # Download JSON File as bytes object and convert to Document Object
            print(f"Fetching {blob.name}")
            document = documentai.Document.from_json(
                blob.download_as_bytes(), ignore_unknown_fields=True
            )

            # For a full list of Document object attributes, please reference this page:
            # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document

            # Read the text recognition output from the processor
            print("The document contains the following text:")
            print(document.text)
            totaldoc=totaldoc+'\n'+document.text


Output files:
Fetching Careplan_output/16475123638674577515/0/Download_Sample-Care-Plan_1-0.json
The document contains the following text:
MedicalDirector®
Sample Care Plan
This care plan has been generated using Medical Director Care - a fully
integrated care planning tool within Medical Director software that combines
patient information with crucial clinical inputs to produce comprehensive
care plans, increase compliance and promote better patient care.
Find out more
HCN Practice Wide Widgets Account MD-Test
205 Bourbong Street, Bundaberg, QLD, 4670
Phone: 1300 788 802 | Fax: 07 4151 1896
TO BE REVIEWED
24/09/2020
PATIENT DETAILS
CAREPLAN Graham ANDREWS
PATIENT'S GOAL
Gender: Male
Date of Birth: 14/03/2000
Social History
Never smoked
CAREPLAN MEMBERS
MS ALISON CAFE
MEDICATIONS
Family History
Dietitian-Paediatric
Drug name
Directions
Condition/s
Created by Dr A Practitioner
205 Bourbong Street, Bundaberg, QLD | 1300 788 802
Address: 2 Kennedy
Road Bundaberg QLD
4670
Phone 07 4923 456

In [13]:
total_doc=totaldoc

### extract Patient entity

In [16]:
#extract entities
from vertexai.generative_models import GenerativeModel
from google.cloud import aiplatform
import json

# Choose the Gemini model (check if available in your project)
model_name = "gemini-2.5-pro"   # or "gemini-1.5-flash-preview" if faster/lighter

generative_model = GenerativeModel(model_name)
 
schema='''
{ 
  "resourceType" : "Patient",  
  "identifier" : [{ Identifier }], // An identifier for this patient
  "active" : <boolean>, // Whether this patient's record is in active use
  "name" : [{
      "text": "",
      "family": "",
      "given": [""]
    }], // A name associated with the patient
  "telecom" : [{
  // from Element: extension
  "system" : "<code>", // I phone | fax | email | pager | url | sms | other
  "value" : "<string>", // I The actual contact point details
  "use" : "<code>", // home | work | temp | old | mobile - purpose of this contact point
  "rank" : "<positiveInt>", // Specify preferred order of use (1 = highest)
  "period" : { Period } // Time period when the contact point was/is in use
}], // A contact detail for the individual
  "gender" : "<code>", // male | female | other | unknown
  "birthDate" : "<date>", // The date of birth for the individual
  // deceased[x]: Indicates if/when the individual is deceased. One of these 2:
  "deceasedBoolean" : <boolean>,
  "deceasedDateTime" : "<dateTime>",
  "address" : [{
  // from Element: extension
  "use" : "<code>", // home | work | temp | old | billing - purpose of this address
  "type" : "<code>", // postal | physical | both
  "text" : "<string>", // Text representation of the address
  "line" : ["<string>"], // Street name, number, direction & P.O. Box etc.
  "city" : "<string>", // Name of city, town etc.
  "district" : "<string>", // District name (aka county)
  "state" : "<string>", // Sub-unit of country (abbreviations ok)
  "postalCode" : "<string>", // Postal code for area
  "country" : "<string>", // Country (e.g. may be ISO 3166 2 or 3 letter code)
  "period" : { Period } // Time period when address was/is in use
}], // An address for the individual
  "maritalStatus" : { 
  // from Element: extension
  "coding" : [{ Coding }], // Code defined by a terminology system
  "text" : "<string>" // Plain text representation of the concept
}, // Marital (civil) status of a patient
  "generalPractitioner" : [{  "<string>" }], // Patient's nominated primary care provider
   
}
'''

# Create prompt
prompt = f"""
You are a clinical data extraction assistant.
Your job is to extract patient information from a clinical document or care plan.
Return the output as a structured FHIR R4 Patient.
⚠ Only include fields that are present or can be inferred from the text. Do not include empty strings, empty arrays ([]) or null values.

follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>

Text:
{total_doc}
"""

# Generate response
response = generative_model.generate_content(prompt)

# Print structured output
print(response.text)

patient=response.text

```json
{
  "resourceType": "Patient",
  "identifier": [
    {
      "type": {
        "text": "Medicare Number"
      },
      "system": "http://ns.electronichealth.net.au/id/medicare-number",
      "value": "3500265121"
    }
  ],
  "active": true,
  "name": [
    {
      "text": "Graham ANDREWS",
      "family": "ANDREWS",
      "given": [
        "Graham"
      ]
    }
  ],
  "telecom": [
    {
      "system": "phone",
      "value": "07 4923 4567",
      "use": "home"
    }
  ],
  "gender": "male",
  "birthDate": "2000-03-14",
  "address": [
    {
      "use": "home",
      "text": "2 Kennedy Road Bundaberg QLD 4670",
      "line": [
        "2 Kennedy Road"
      ],
      "city": "Bundaberg",
      "state": "QLD",
      "postalCode": "4670"
    }
  ],
  "generalPractitioner": [
    {
      "display": "Dr A Practitioner"
    }
  ]
}
```


### extract Condition entities

In [17]:
#extract entities
from vertexai.generative_models import GenerativeModel
from google.cloud import aiplatform
import json

# Choose the Gemini model (check if available in your project)
model_name = "gemini-2.5-pro"   # or "gemini-1.5-flash-preview" if faster/lighter

generative_model = GenerativeModel(model_name)


#prompt = f"Extract clinical entities from the following text:\n{document.text}"

schema='''
{
  "id": "",
  "identifier": [{
    "system": "",
    "value": ""
  }],
  "meta": {
    "source": "",
    "tag": [{
      "code": "",
      "display": "",
      "system": ""
    }]
  },
  "clinicalStatus": {
    "coding": [{
      "code": "",
      "display": "",
      "system": ""
    }]
  },
  "verificationStatus": {
    "coding": [{
      "code": "",
      "display": "",
      "system": ""
    }]
  },
  "category": [{
    "coding": [{
      "code": "",
      "display": "",
      "system": ""
    }],
    "text": ""
  }],
  "code": {
    "coding": [{
      "code": "",
      "display": "",
      "system": ""
    }],
    "text": ""
  },
  "bodySite": [{
    "coding": [{
      "code": "",
      "display": "",
      "system": ""
    }],
    "text": ""
  }],
  "subject": {
    "reference": ""
  },
  "encounter": {
    "reference": ""
  },
  "onsetDateTime": "",
  "onsetPeriod": {
    "start": "",
    "end": ""
  },
  "recordedDate": "",
  "asserter": {
    "display": "",
    "reference": ""
  },
  "recorder": {
    "display": "",
    "reference": ""
  },
  "note": [{
    "text": ""
  }]
}


'''

# Create prompt
prompt = f"""
You are a clinical data extraction assistant.
Your job is to extract patient information from a clinical document or care plan.
Return the output as a structured FHIR R4 Condition.
⚠ Only include fields that are present or can be inferred from the text. Do not include empty strings, empty arrays ([]) or null values.

follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>
each condition should be separated from the other one using '----------'
Text:
{total_doc}
"""

# Generate response
response = generative_model.generate_content(prompt)

# Print structured output
print(response.text)

condition=response.text

```json
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
      }
    ]
  },
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/condition-category",
          "code": "problem-list-item",
          "display": "Problem List Item"
        }
      ]
    }
  ],
  "code": {
    "text": "Ear foreign body removal"
  },
  "subject": {
    "reference": "Patient/GrahamANDREWS"
  },
  "onsetDateTime": "2002",
  "recordedDate": "2020-06-24",
  "asserter": {
    "display": "Dr A Practitioner"
  }
}
```
----------
```json
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active",
        "display": "Active"
      }
    ]
  },
  "verificationStatus": {
    "co

### extract Careplan entity

In [25]:
#extract entities
from vertexai.generative_models import GenerativeModel
from google.cloud import aiplatform
import json

# Choose the Gemini model (check if available in your project)
model_name = "gemini-2.5-pro"   # or "gemini-1.5-flash-preview" if faster/lighter

generative_model = GenerativeModel(model_name)


#prompt = f"Extract clinical entities from the following text:\n{document.text}"

schema='''
{
  "id": "",
  "intent":"plan",
  "status": "",
  "title": "",
  "description": "",
  "created": "",
  "period": {
    "start": "",
    "end": ""
  },
  "subject": {
    "reference": ""
  },
  "author": {
    "display": ""    
  },
  "category": [
    {
      "coding": [
        {
          "code": "",
          "display": "",
          "system": ""
        }
      ],
      "text": ""
    }
  ],
  "addresses": [
    {
      "display": "",
      "reference": ""
    }
  ],
  "activity": [
    {
      "detail": {
        "description": ""
        "status":"in-progress"
      },
      "progress": [
        {
          "text": ""
        }
      ]
    }
  ],
  "note": [
    {
      "text": ""
    }
  ]
}


'''


# Create prompt
prompt = f"""
You are a clinical data extraction assistant.
Your job is to extract patient information from a clinical document or care plan.
Return the output as a structured FHIR R4 CarePlan.
⚠ Only include fields that are present or can be inferred from the text. Do not include empty strings, empty arrays ([]) or null values.

follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>

Text:
{total_doc}
"""

# Generate response
response = generative_model.generate_content(prompt)

# Print structured output
print(response.text)

careplan=response.text

```json
{
  "id": "careplan-graham-andrews",
  "intent": "plan",
  "status": "active",
  "title": "CAREPLAN Graham ANDREWS",
  "description": "Comprehensive care plan for Graham Andrews focusing on lifestyle, biomedical, pharmacological, cardiovascular, and obesity management.",
  "created": "2020-06-24",
  "period": {
    "start": "2020-06-24",
    "end": "2020-09-24"
  },
  "subject": {
    "reference": "Patient/graham-andrews",
    "display": "Graham ANDREWS"
  },
  "author": {
    "display": "Dr A Practitioner"
  },
  "category": [
    {
      "text": "Lifestyle (General Health)"
    },
    {
      "text": "Biomedical (General Health)"
    },
    {
      "text": "Pharmacological (General Health)"
    },
    {
      "text": "Cardiovasular Vascular Disease"
    },
    {
      "text": "Obesity"
    }
  ],
  "addresses": [
    {
      "display": "Ant Bite Allergy"
    },
    {
      "display": "Obesity"
    },
    {
      "display": "Cardiovascular Disease"
    }
  ],
  "activity": [
 

### store entites in FHIR data store

In [20]:
#format enities in fhir
import re
import json
import uuid
from datetime import datetime

def func_extract_json(text):
    # Extract text from Gemini
    output = text

    # Try to extract JSON block using regex
    match = re.search(r'\{.*\}', output, re.DOTALL)
    if match:
        json_str = match.group(0)
        entities = json.loads(json_str)
    else:
        raise ValueError("No valid JSON found in LLM response")
    return entities
 
#strore result in fhir store
from google.oauth2 import service_account
from googleapiclient.discovery import build
import json

# credentials_path = "path/to/your-service-account.json"

# # Authenticate
# scopes = ["https://www.googleapis.com/auth/cloud-platform"]
# credentials = service_account.Credentials.from_service_account_file(
#     credentials_path, scopes=scopes
# )

from google.auth import default
from googleapiclient.discovery import build

credentials, project_id = default()
import google.auth
from google.auth.transport.requests import Request

creds, project_id = google.auth.default()
creds.refresh(Request())

print("✅ Access token:", creds.token[:20], "...")  # Just to confirm it's working


service = build("healthcare", "v1", credentials=credentials) 

 
import google.auth
from google.auth.transport.requests import AuthorizedSession

credentials, project_id = google.auth.default()
authed_session = AuthorizedSession(credentials)


def upload_fhir_resource(resource):

    resource_type = resource["resourceType"]
    print(f"Uploading {resource_type}:\n{json.dumps(resource, indent=2)}")

    try:
       #this is also correct
        response = authed_session.post(
            #f"https://healthcare.googleapis.com/v1/{fhir_store}/fhir/Patient",
             f"https://healthcare.googleapis.com/v1/{fhir_store}/fhir/{resource_type}",
            json=resource
        )
        response.raise_for_status()
        created_patient = response.json()
        response=created_patient
        print("Created Patient:", created_patient)

    except Exception as e:
        print(f"❌ Failed to upload {resource_type}:\n{e}")
    return response



✅ Access token: ya29.c.c0ASRK0GZKSaI ...


In [21]:
fhir_patient=func_extract_json(patient)
fhir_patient['resourceType']='Patient'
try:
    fhir_patient.pop('id') 
except:
    s=''
res=upload_fhir_resource(fhir_patient)


Uploading Patient:
{
  "resourceType": "Patient",
  "identifier": [
    {
      "type": {
        "text": "Medicare Number"
      },
      "system": "http://ns.electronichealth.net.au/id/medicare-number",
      "value": "3500265121"
    }
  ],
  "active": true,
  "name": [
    {
      "text": "Graham ANDREWS",
      "family": "ANDREWS",
      "given": [
        "Graham"
      ]
    }
  ],
  "telecom": [
    {
      "system": "phone",
      "value": "07 4923 4567",
      "use": "home"
    }
  ],
  "gender": "male",
  "birthDate": "2000-03-14",
  "address": [
    {
      "use": "home",
      "text": "2 Kennedy Road Bundaberg QLD 4670",
      "line": [
        "2 Kennedy Road"
      ],
      "city": "Bundaberg",
      "state": "QLD",
      "postalCode": "4670"
    }
  ],
  "generalPractitioner": [
    {
      "display": "Dr A Practitioner"
    }
  ]
}
Created Patient: {'active': True, 'address': [{'city': 'Bundaberg', 'line': ['2 Kennedy Road'], 'postalCode': '4670', 'state': 'QLD', 'text

In [ ]:
id='0a8212cc-ad5e-40e0-b47f-14760116b0cc'

In [22]:
for cndtion in condition.split('----------'):    
    if (cndtion.strip()!=''):
            try:
                print(cndtion)            
                fhir_condition=func_extract_json(cndtion)
                #fhir_condition['identifier']=f"{res['id']}"
                try:
                    fhir_condition['subject']['reference']=f"Patient/{res['id']}"
                except:
                    fhir_condition['subject'] = {"reference": f"Patient/{res['id']}"}

                fhir_condition['resourceType']='Condition'
                #fhir_condition.pop('id')
                resccnd=upload_fhir_resource(fhir_condition)
            except:
                continue
         

```json
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
      }
    ]
  },
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/condition-category",
          "code": "problem-list-item",
          "display": "Problem List Item"
        }
      ]
    }
  ],
  "code": {
    "text": "Ear foreign body removal"
  },
  "subject": {
    "reference": "Patient/GrahamANDREWS"
  },
  "onsetDateTime": "2002",
  "recordedDate": "2020-06-24",
  "asserter": {
    "display": "Dr A Practitioner"
  }
}
```

Uploading Condition:
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
      }
    ]
  },
  "category": [
    {
    

In [27]:
print(careplan)

```json
{
  "id": "careplan-graham-andrews",
  "intent": "plan",
  "status": "active",
  "title": "CAREPLAN Graham ANDREWS",
  "description": "Comprehensive care plan for Graham Andrews focusing on lifestyle, biomedical, pharmacological, cardiovascular, and obesity management.",
  "created": "2020-06-24",
  "period": {
    "start": "2020-06-24",
    "end": "2020-09-24"
  },
  "subject": {
    "reference": "Patient/graham-andrews",
    "display": "Graham ANDREWS"
  },
  "author": {
    "display": "Dr A Practitioner"
  },
  "category": [
    {
      "text": "Lifestyle (General Health)"
    },
    {
      "text": "Biomedical (General Health)"
    },
    {
      "text": "Pharmacological (General Health)"
    },
    {
      "text": "Cardiovasular Vascular Disease"
    },
    {
      "text": "Obesity"
    }
  ],
  "addresses": [
    {
      "display": "Ant Bite Allergy"
    },
    {
      "display": "Obesity"
    },
    {
      "display": "Cardiovascular Disease"
    }
  ],
  "activity": [
 

In [28]:

fhir_careplan=func_extract_json(careplan)

fhir_careplan['subject']['reference']=f"Patient/{res['id']}"
fhir_careplan['resourceType']='CarePlan'
try:
    fhir_careplan.pop('id') 
except:
    s=''
rescp=upload_fhir_resource(fhir_careplan)

            

Uploading CarePlan:
{
  "intent": "plan",
  "status": "active",
  "title": "CAREPLAN Graham ANDREWS",
  "description": "Comprehensive care plan for Graham Andrews focusing on lifestyle, biomedical, pharmacological, cardiovascular, and obesity management.",
  "created": "2020-06-24",
  "period": {
    "start": "2020-06-24",
    "end": "2020-09-24"
  },
  "subject": {
    "reference": "Patient/0a8212cc-ad5e-40e0-b47f-14760116b0cc",
    "display": "Graham ANDREWS"
  },
  "author": {
    "display": "Dr A Practitioner"
  },
  "category": [
    {
      "text": "Lifestyle (General Health)"
    },
    {
      "text": "Biomedical (General Health)"
    },
    {
      "text": "Pharmacological (General Health)"
    },
    {
      "text": "Cardiovasular Vascular Disease"
    },
    {
      "text": "Obesity"
    }
  ],
  "addresses": [
    {
      "display": "Ant Bite Allergy"
    },
    {
      "display": "Obesity"
    },
    {
      "display": "Cardiovascular Disease"
    }
  ],
  "activity": [
  